In [1]:
from systemtools.hayj import *
from systemtools.location import *
from systemtools.basics import *
from systemtools.file import *
from systemtools.printer import *
from databasetools.mongo import *
from newstools.goodarticle.utils import *
from nlptools.preprocessing import *
from nlptools.news import parser as newsParser
from machinelearning.iterator import *
from twinews.utils import *

In [2]:
from nltk.stem import WordNetLemmatizer
from nlptools.preprocessing import *
from nlptools.basics import *

In [3]:
import pandas as pd
import json
import pymongo
import csv

In [4]:
ConnUsers = getUsersCollection()

twinews users (version 1.0) initialised.


In [5]:
ConnNews = getNewsCollection()

twinews news (version 1.0) initialised.


# json dump

In [9]:
def readMongo(con,myquery):
    output = []
    try:
        #myquery = {} # 查询条件
        for item in con.find(myquery):
            del item['_id']
            output.append(item)
        print ('data reading finished')
    except Exception as e:
        print (e)
    
    return output


In [10]:
def getJson(con, myquery,table_name):
    dataDir = '/home/yuting/PycharmProjects/data'
    data = readMongo(con,myquery)
    with open(dataDir + "/" + table_name + '.json', 'w', encoding="UTF-8") as jf:
        jf.write(json.dumps(data, indent=2))    

In [ ]:
getJson(ConnNews,{},'news')

data reading finished


In [41]:
getJson(ConnUsers,{{},{"user_id":1},{"news":1}},'users_test')

data reading finished


In [29]:
logger = Logger(tmpDir('logs') + "/dssm_yf.log")
tt = TicToc(logger=logger)
tt.tic()

--> tictoc starts...


-1

# get training data urls

In [6]:
userHistory = {}
negativeNewList = []

# read user with more than 30 news
for row in ConnUsers.find().limit(100):
    if len(row["news"]) >= 30:
        userHistory[row["user_id"]] = row["news"]
# get news for negative sampling     
for row in ConnNews.find().limit(1000):
    negativeNewList.append(row["url"])

In [26]:
dataUrls = [] # composed of user_id, 20 history as query, 10 history as positive doc, 10 from neg as negative doc

for k in userHistory.keys():
    query20 = random.sample(userHistory[k],20)
    doc10 = random.sample(list(set(userHistory[k]).difference(set(query20))),10)
    dataUrls.append([k,query20,doc10])
    
assert len(negativeNewList)//len(userHistory.keys()) >= 10   #make sure we have enough negative sample

for k in range(len(dataUrls)):
    try:
        negTemp = []
        #if len(negativeNewList) > 10:
        negTemp = random.sample(list(set(negativeNewList).difference(set(userHistory[dataUrls[k][0]]))),10)
        dataUrls[k].append(negTemp)
        negativeNewList = list(set(negativeNewList)^set(negTemp))
    except Exception as e:
        print(e)

# to csv

In [275]:
with open('/home/yuting/PycharmProjects/data/dssm_test', 'w', newline='') as csvfile:
    writer  = csv.writer(csvfile)
    for row in dataUrls:
        writer.writerow(row)

# sampling and labeling

In [27]:
queryList = []
posList = []
negList = []

for i in range(len(dataUrls)):
#     queryList.append(dataUrls[i][1])
#     posList.append(dataUrls[i][2])
#     negList.append(dataUrls[i][3])
    for j in range(len(dataUrls[i][1])):
        queryList.append(dataUrls[i][1][j])
    for k in range(len(dataUrls[i][2])):
        posList.append(dataUrls[i][2][k])
    for m in range(len(dataUrls[i][3])):
        negList.append(dataUrls[i][3][m])
urlsList = queryList + posList + negList

In [30]:
sentences = getNewsSentences(urlsList, logger=logger)
bp(sentences, logger=logger)

twinews news (version 1.0) initialised.
  0% [                    ]
 10% [==                  ] (1m 48.989s left)
 20% [====                ] (1m 35.479s left)
 30% [======              ] (1m 25.96s left)
 40% [========            ] (1m 14.264s left)
 50% [==========          ] (1m 1.27s left)
 60% [============        ] (48.98s left)
 70% [==============      ] (37.118s left)
 80% [================    ] (24.327s left)
 90% [==================  ] (11.874s left)
100% [====================] (total duration: 1m 56.5s, mean duration: 0.044s)
[
  [ [ Media, playback, ..., Manhattan, . ], [ A, -, ..., arrested, . ], ..., [ Please, include, ..., journalist, . ], [ You, can, ..., ways, : ] ],
  [ [ Now, ,, ..., long, . ], [ After, more, ..., life, . ], ..., [ ", But, ..., schedule, . ], [ Not, when, ..., calling, . ] ],
  ...,
  [ [ TAMPA, ,, ..., morning, . ], [ The, Hillsborough, ..., Times, . ], ..., [ Email, __email__, ..., feed, . ], [ Chart, :, ..., school, year ] ],
  [ [ (, WSAV, ..., 

In [31]:
for i in range(len(sentences)):
    sentences[i] = flattenLists(sentences[i])
docs = sentences
bp(docs, logger)

[ [ Media, playback, ..., ways, : ], [ Now, ,, ..., calling, . ], ..., [ TAMPA, ,, ..., school, year ], [ (, WSAV, ..., sick, . ] ]


In [32]:
if True:
    for i in pb(list(range(len(docs))), logger=logger, message="Lower casing"):
        for u in range(len(docs[i])):
            docs[i][u] = docs[i][u].lower()
bp(docs, logger)

Lower casing   0% [                    ]
Lower casing  10% [==                  ] (0.809s left)
Lower casing  20% [====                ] (0.639s left)
Lower casing  30% [======              ] (0.536s left)
Lower casing  40% [========            ] (0.464s left)
Lower casing  50% [==========          ] (0.38s left)
Lower casing  60% [============        ] (0.306s left)
Lower casing  70% [==============      ] (0.227s left)
Lower casing  80% [================    ] (0.144s left)
Lower casing  90% [==================  ] (0.067s left)
Lower casing 100% [====================] (total duration: 0.64s, mean duration: 0s)
[ [ media, playback, ..., ways, : ], [ now, ,, ..., calling, . ], ..., [ tampa, ,, ..., school, year ], [ (, wsav, ..., sick, . ] ]


In [33]:
if True:
    lemmatizer = WordNetLemmatizer()
    pbar = ProgressBar(len(docs), logger=logger, message="Lemmatization")
    for i in range(len(docs)):
        for u in range(len(docs[i])):
            docs[i][u] = lemmatizer.lemmatize(docs[i][u])
        pbar.tic()
bp(docs, logger)

Lemmatization   0% [                    ]
Lemmatization  10% [==                  ] (17.55s left)
Lemmatization  20% [====                ] (11.08s left)
Lemmatization  30% [======              ] (8.26s left)
Lemmatization  40% [========            ] (6.645s left)
Lemmatization  50% [==========          ] (5.16s left)
Lemmatization  60% [============        ] (4s left)
Lemmatization  70% [==============      ] (2.91s left)
Lemmatization  80% [================    ] (1.825s left)
Lemmatization  90% [==================  ] (0.848s left)
Lemmatization 100% [====================] (total duration: 7.96s, mean duration: 0.003s)
[ [ medium, playback, ..., way, : ], [ now, ,, ..., calling, . ], ..., [ tampa, ,, ..., school, year ], [ (, wsav, ..., sick, . ] ]


In [34]:
docs = filterCorpus(docs, minDF=1/2000, maxDF=300,
                    removeEmptyDocs=False, allowEmptyDocs=False, logger=logger)
for doc in docs: assert len(doc) > 0
bp(docs, logger)

Voc removed because of minDF (21854 elements):
{ #eaglesparade, 0, 0comments, 100bn, 100gm, 103rd, 1040x, 105mt, 109th, 10am, ..., zwillich, zwonitzer, zynga, zyprexa, zyxel, 😁, 😂, 😉, 😒, 😢 }
Voc removed because of maxDF (300 elements):
{ ", ', (, ), ,, -, ., :, ;, ?, ..., woman, work, working, world, would, year, yet, york, you, your }
44.09% of voc will be removed.
[ [ medium, playback, ..., contact, following ], [ spirited, race, ..., ranch, calling ], ..., [ tampa, fla., ..., salary, county ], [ wsav, hilton, ..., drop, sick ] ]


In [39]:
a = [['sdf','dfwer','dfs'],['derw','ewrc5']]
a[1] = ' '.join(a[1])

In [40]:
a

[['sdf', 'dfwer', 'dfs'], 'derw ewrc5']

In [35]:
def toStr(sentencesList):
    for i in range(len(sentencesList)):
        sentencesList[i] = ' '.join(sentencesList[i])
    return sentencesList

In [36]:
corpus = toStr(docs)
for corp in corpus: assert len(corp) > 0
bp(corpus,logger)

[
  medium playback unsupported device medium caption terror attack event unfolded eight killed seriousl,
  spirited race succeed underway talk turned mr. brown chapter resilience imprint identified mr. brown,
  ...,
  tampa fla. hundred jefferson student walked class protested teacher salary increase wednesday mornin,
  wsav hilton head robotics team officially renowned team won international robotics competition mit t
]


In [38]:
c = ' '.join(corpus)
with open('/home/yuting/PycharmProjects/data/dssm_test_corpus.txt', 'a') as f:
    f.write(c) 

In [54]:
urlDocs = dict()
for i in range(len(urlsList)):
    urlDocs[urlsList[i]] = corpus[i]

bp(urlDocs, 2, logger)

{
  http://a.msn.com/00/en-us/BBFZDuB?ocid=st: stock went wild ride friday falling sharply michael flynn directed talk russian abc reported flynn s,
  http://a.msn.com/01/en-us/BBIVxj7?ocid=st: worker ice deport immigrant quit courtesy daisy robert quit helena mont. robert immediately reservat,
  http://a.msn.com/0B/en-us/AAuMl9k?ocid=st: widow eagle guitarist frey sue nyc hospital wrongful death popular breakup song michael loccisano ge,
  http://abc6.wjbf.com/2AEprPm: drew foundation host night drew copyright wjbf reserved video augusta ga. wjbf drew legacy baseball ,
  http://abc6.wjbf.com/2CgdJM1: evans host 12th annual monterrey christmas tournament copyright wjbf reserved video evans ga. wjbf e,
  ...,
  https://www.yourtango.com/experts/shireen-yates/safe-holiday-eating-tips-while-gluten-free-diet: communicating dietary advance alleviate tension everyone involved holiday packed social gathering pl,
  https://www.zerohedge.com/news/2017-01-02/currency-collapse-roman-empire: peak 

In [55]:
h = ["sentence1","sentence2","label"]
with open('/home/yuting/PycharmProjects/data/dssm_test_train.csv', 'w', newline='') as csvfile:
    writer  = csv.writer(csvfile)
    writer.writerow(h)
    for i in range(len(dataUrls)):
        query = ""
        for j in range(len(dataUrls[i][1])):
            query += urlDocs[dataUrls[i][1][j]]
            for k in range(len(dataUrls[i][2])):
                posSamp = []
                pos = urlDocs[dataUrls[i][2][k]]
                posSamp.append(query)
                posSamp.append(pos)
                posSamp.append("1")
                writer.writerow(posSamp)
            for m in range(len(dataUrls[i][3])):
                negSamp = []
                neg = urlDocs[dataUrls[i][3][m]]
                negSamp.append(query)
                negSamp.append(neg)
                negSamp.append("0")
                writer.writerow(negSamp)
    

In [5]:
h = ["sentence1","sentence2","label"]
with open('/home/yuting/PycharmProjects/data/dssm_test_train.csv','rb') as reader, open('/home/yuting/PycharmProjects/data/dssm_test_train_sub.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(h)
    for index, line in enumerate(reader):
        if (index-1) % 10 == 0:
            writer.writerow(line)

In [31]:
h = "sentence1"+','+"sentence2"+','+"label"+'\n'
with open('/home/yuting/PycharmProjects/data/dssm_test_dev.csv','r') as reader:
    with open('/home/yuting/PycharmProjects/data/dssm_test_dev_sub.txt','w') as writer:
        writer.write(h)
        for index, line in enumerate(reader):
            if (index-1)%10==0:
                writer.write(line)

# word hashing

In [17]:
import csv

In [18]:
dir(csv)

['Dialect',
 'DictReader',
 'DictWriter',
 'Error',
 'OrderedDict',
 'QUOTE_ALL',
 'QUOTE_MINIMAL',
 'QUOTE_NONE',
 'QUOTE_NONNUMERIC',
 'Sniffer',
 'StringIO',
 '_Dialect',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '__version__',
 'excel',
 'excel_tab',
 'field_size_limit',
 'get_dialect',
 'list_dialects',
 're',
 'reader',
 'register_dialect',
 'unix_dialect',
 'unregister_dialect',
 'writer']

# extra note

In [106]:
list(set(b).difference(set(a)))

[9, 4, 5]

In [19]:
filename = open('/home/yuting/PycharmProjects/data/dssm_test_dev.csv')
reader = csv.reader(filename)
i = 0
for row in reader:
    corpus = ' '.join(row)
    print(corpus)
    i += 1
    if i >= 2:
        break

sentence1 sentence2 label
challenge team unveiling innovation tackle glass ceiling along apps wearable tech necklace team created eye catching art installation happened road tested idea silicon valley bbc influential inspirational challenging tackle biggest facing glass ceiling female illiteracy harassment space sexism sport coming real solution involved idea facebook instagram twitter 100women student facing hate crime charge admitting smearing bodily fluid black roommate possession apparent attempt force room brianna brochu charged misdemeanour criminal mischief breach peace police west hartford connecticut judge add felony bigotry charge greg woodward m brochu longer student campaign roommate chennel rowe apparently light via instagram bragged action published local described m rowe facebook video reportedly wrote finally yo girl rid roommate ! spitting coconut oil putting moldy clam dip lotion ... putting toothbrush sun shine finally goodbye jamaican barbie addressed student mr woo